In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch import tensor
from torch.autograd import Variable
from generate_data import generate_data, data_loader

In [126]:
batch_size = 32

lr = 0.0001

loss_fn = nn.CrossEntropyLoss()

In [127]:
N = 4*int(1e4)

X, y = generate_data(N, is_short=True, type = 2)

X_test, y_test = generate_data(2000, is_short=True, type = 2)

In [128]:
class RNN_predictor(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(RNN_predictor, self).__init__()
        
        # Number of hidden dimensions
        self.hidden_dim = hidden_dim
        
        # Number of hidden layers
        self.layer_dim = layer_dim
        
        # RNN
        self.rnn = nn.RNN(input_dim, hidden_dim, layer_dim, nonlinearity='relu')
        
        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)
    
        self.activation = nn.Softmax()
    def forward(self, x):
        
        # Initialize hidden state with zeros
        h0 = Variable(torch.zeros(self.layer_dim, x.shape[1], self.hidden_dim))
            
        # One time step
        out, hn = self.rnn(x, h0)
        out = self.fc(out[ -1,:, :])
        return self.activation(out)


model = RNN_predictor(3, 6, 1, 2)

In [129]:
print(model)

RNN_predictor(
  (rnn): RNN(3, 6)
  (fc): Linear(in_features=6, out_features=2, bias=True)
  (activation): Softmax(dim=None)
)


In [133]:
def test(model):
    model.eval()
    test_batches = data_loader(X_test,y_test,batch_size=batch_size, min_length = 20)
    acc = 0.0
    i = 0
    for data, labels in test_batches:
        pred = model(data).view((-1,2))
        corr = (torch.argmax(pred,1) == torch.argmax(labels,1)).float().mean().item()
        acc += corr
        i += 1
    return acc/i

def train(model,n_epochs):
    parameters = model.parameters()

    optimizer = torch.optim.Adam(parameters, lr=lr)

    model.train()


    for i in range(n_epochs):
        model.train()
        train_batches = data_loader(X,y,batch_size=batch_size, min_length = 20)

        sum_loss = 0.0
        i_loss = 0
        for data, labels in train_batches:
            pred = model(data)
            loss_fn = F.binary_cross_entropy
            loss = loss_fn(pred,labels)
            loss.backward()
            optimizer.step()

            sum_loss += loss.item()
            i_loss += 1
        print("loss",sum_loss/i_loss)
        acc = test(model)
        print("acc",acc)


train(model,80)

c:\Users\jakob\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


loss 0.5689751416444778
acc 0.6300403225806451
loss 0.5658170771121979
acc 0.7298387096774194
loss 0.56370882229805
acc 0.7288306451612904
loss 0.5762949701070785
acc 0.6970766129032258
loss 0.5809571995019913
acc 0.7132056451612904
loss 0.5695738674163818
acc 0.7137096774193549
loss 0.5743399885177612
acc 0.5866935483870968
loss 0.5679350829839707
acc 0.7570564516129032
loss 0.5650583145618439
acc 0.6144153225806451
loss 0.5768949269294739
acc 0.7580645161290323
loss 0.5730006489038467
acc 0.6441532258064516
loss 0.5671577861309052
acc 0.6431451612903226
loss 0.5686790298938751
acc 0.7157258064516129
loss 0.5806645032167435
acc 0.6970766129032258


KeyboardInterrupt: 

In [134]:
torch.save(model.state_dict(), "RNN_model0_dict_SIZE6_40epochs")
